## Imports

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.datasets
from cvxpy import *

## 1- Primal Formulation for Soft-Margin SVM

In [15]:
# Load Data
df = pd.read_csv('svm_train.txt', delim_whitespace=True)
X = df.drop('y', axis=1).values
y = df['y'].values
n = X.shape[0]

In [16]:
epsilon = Variable(n, nonneg = True)
w = Variable(X.shape[1])
b = Variable(1)
C = 1

constraints = []
for i in range(n):
    constraints += [y[i] * (w[0] * X[i][0] + w[1] * X[i][1] + b) >= 1 - epsilon[i]]
objective = Minimize(0.5 * norm(w,2)**2 + C * sum(epsilon))
prob = Problem(objective, constraints)
result = prob.solve(solver=ECOS)

In [17]:
print('optimal w=', w.value)
print('optimal b=', b.value[0])

optimal w= [-1.00209063  3.64672864]
optimal b= -1.8672371699696897


## 2-Dual Formulation of Soft-Margin SVM

In [18]:
alpha = Variable(n, nonneg = True)
C = 1
P = atoms.affine.wraps.psd_wrap(X @ X.T)
Y = diag(y)

dual_constraints = [alpha <= C, sum(y * alpha) == 0]

dual_objective = Maximize(sum(alpha) - 0.5 * quad_form(alpha @ Y, P) / C)

dual_prob = Problem(dual_objective, dual_constraints)
dual_result = dual_prob.solve()
# dual_constraints = 

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\cvxpy\expressions\expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)


In [19]:
# new_alpha = [i if i > 10**-10 else 0 for i in alpha.value]
w = 0
for i in range(n):
    w += alpha.value[i] * y[i] * X[i]
print(dual_result)

746.2681355070266


In [20]:
print('optimal w=', w)

optimal w= [-1.00209063  3.64672864]


In [32]:
b = []
for i in range(n):
    b.append(y[i] - w[0] * X[i][0] - w[1] * X[i][1])
    # b.append(y[i] - multiply(w.value, X[i]))
print('optimal b=', np.mean(b))

optimal b= -1.915074411165263
